In [10]:
import datasets

In [9]:
#carrega natural questions

from datasets import load_dataset
dataset = load_dataset('json', data_files="simplified-nq-train.jsonl")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 11.65it/s]
Generating train split: 307373 examples [02:19, 2210.56 examples/s]


In [11]:
#carrega modelo de tradução

from transformers import pipeline
pipe = pipeline("translation", model="Helsinki-NLP/opus-mt-tc-big-en-pt")


2023-11-29 14:02:15.199794: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-29 14:02:15.422451: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-29 14:02:16.218130: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [12]:
df = pd.DataFrame(columns = ['title', 'context','question', 'answer'])

In [13]:
sample = dataset['train'][0]

In [14]:
titulo = sample['document_text']

In [15]:
re.findall(r'(?<=<h1>)(.+?)(?=</h1>)', titulo, re.IGNORECASE)[0]

NameError: name 're' is not defined

In [ ]:
sample['annotations']

[{'yes_no_answer': 'NONE',
  'long_answer': {'start_token': 1952,
   'candidate_index': 54,
   'end_token': 2019},
  'short_answers': [{'start_token': 1960, 'end_token': 1969}],
  'annotation_id': 5.931654502200276e+17}]

In [ ]:
doc = sample['document_text']

In [ ]:
sample['annotations'][0]['long_answer']

{'start_token': 1952, 'candidate_index': 54, 'end_token': 2019}

In [ ]:
init = sample['annotations'][0]['short_answers'][0]['start_token']
end = sample['annotations'][0]['short_answers'][0]['end_token']

In [ ]:
" ".join(doc.split(" ")[init:end])

"a newsletter sent to an advertising firm 's customers"

In [ ]:
#sample['question_text']

'what happened to the lost settlement of roanoke'

In [16]:
import pandas as pd
import re
import sys
import time

In [22]:
def translate(size=10, dataset=dataset):
    df = pd.DataFrame(columns = ['title', 'question', 'long_answer', 'short_answer'])
    small_set = dataset['train']
    for i in range(size):

        doc = re.sub(r'''<(?:"[^"]*"['"]*|'[^']*'['"]*|[^'">])+>''','',small_set[i]['document_text'])
        init_long = small_set[i]['annotations'][0]['long_answer']['start_token']
        end_long = small_set[i]['annotations'][0]['long_answer']['end_token']
        if len(small_set[i]['annotations'][0]['short_answers']) != 0:
            init_short = small_set[i]['annotations'][0]['short_answers'][0]['start_token']
            end_short = small_set[i]['annotations'][0]['short_answers'][0]['end_token']
        else:
            init_short = 0
            end_short  = 0
        question = small_set[i]['question_text']
        long_answer = " ".join(doc.split(" ")[init_long:end_long])
        short_answer = " ".join(doc.split(" ")[init_short:end_short])
        title = re.findall(r'(?<=<h1>)(.+?)(?=</h1>)', small_set[i]['document_text'], re.IGNORECASE)[0]
        #if len(long_answer) > 0:  
            #title = pipe('>>por<< ' + title)[0]['translation_text']
            #long_answer= pipe('>>por<< ' + long_answer)[0]['translation_text']
            #if len(short_answer) != 0:
            #    short_answer= pipe('>>por<< ' + short_answer)[0]['translation_text']
            #question= pipe('>>por<< ' + question)[0]['translation_text']

            
            
        #else:
        #    pass
            
        new_row = {'title': [title], 'question': [question], 'long_answer': [long_answer], 'short_answer': [short_answer]}
        df_aux = pd.DataFrame(new_row)
        df = pd.concat([df, df_aux])

        #configuração do terminal
        sys.stdout.write(f'\rProgress : {((i+1)/size)*100}%')
        sys.stdout.flush()
        
    df = df.loc[df['long_answer'] != '']
    df.reset_index(drop=True, inplace=True)
    df
    return df
    
translate(1000, dataset=dataset).to_excel('saida.xlsx')


Progress : 100.0%99999999999%%

In [ ]:
%%time
traduzido = translate(size=20)

Progress : 100.0%00000000001%Wall time: 1min 51s


In [ ]:
squad = {
    "version": "0.1",
    "data": [
        
    ],
}

In [ ]:
import json
for passage in json.loads(traduzido.to_json(orient='records')):
    estrutura = {
        "title": passage['title'],
        "paragraphs": [
            {
            "context": passage['long_answer'],
            "qas": [
                {
                    "id":0,
                    "question": passage['question'],
                    "answers": [
                        {
                        "answer_start":0,
                        'text':passage['short_answer'],
                        }
                        
                    ],
                },
            ],
            },
        ],
    }
    squad['data'].append(estrutura)
    

In [ ]:
json_data =squad
file_path = "data.json"
with open(file_path, "w") as json_file:
    json.dump(json_data, json_file, indent=4)

print(f"Variable saved as JSON")

Variable saved as JSON


In [23]:
import pandas as pd
import json

# Load the Excel spreadsheet
planilha = pd.read_excel('res.xlsx')

print("Primeira linha da planilha:")
print(planilha.iloc[0])

# Initialize the JSON structure
squad = {"data": []}

# Iterate over the rows of the spreadsheet
for idx, linha in planilha.iterrows():
    estrutura = {
        "title": str(linha['t']),  # Assuming there is a column named 't' for title
        "paragraphs": [
            {
                "context": str(linha['q']),  # Replace 'q' with the correct column name for the question
                "qas": [
                    {
                        "id": idx,
                        "question": str(linha['l']),  # Replace 's' with the correct column name for the answer
                        "answers": [
                            {
                                "answer_start": 0,
                                "text": str(linha['s']),
                            },
                        ],
                    },
                ],
            },
        ],
    }
    squad['data'].append(estrutura)

# Save the JSON to a file
with open('googleTrad2.json', 'w', encoding='utf-8') as json_file:
    json.dump(squad, json_file, ensure_ascii=False, indent=2)


Primeira linha da planilha:
Unnamed: 0                                                      0
title                                            Email marketing 
question        which is the most common use of opt-in e-mail ...
long_answer      A common example of permission marketing is a...
short_answer    a newsletter sent to an advertising firm 's cu...
t                                              Marketing de email
q               que é o uso mais comum do marketing de e-mail ...
l                Um exemplo comum de marketing de permissão é ...
s               Um boletim informativo enviado aos clientes de...
Name: 0, dtype: object
